In [9]:
import numpy as np
import scipy.stats
from random import seed
from itertools import combinations

seed(20)

## Intento con programación funcional

Consulta sobre la máxima densidad de empaquetamiento de esferas en una caja:

https://en.wikipedia.org/wiki/Sphere_packing

La distribución de Maxwell de velocidades en 3D:
$$f(v) = 4\pi \bigg(\frac{m}{2\pi k_B T}\bigg)^{3/2} v^2 e^{-\frac{mv^2}{2k_BT}}$$

### Clases

In [10]:
# Se define una distribución de Maxwell 3D
class Maxwell_3d(scipy.stats.rv_continuous):
  def _pdf(self, x, m, kT):
    down = 2*kT
    return 4*np.pi*((m/(np.pi*down))**1.5)*(x**2)*np.exp(-m*(x**2)/down)

maxwell_3d = Maxwell_3d(shapes='m, kT')

### Funciones

#### Interacciones

In [11]:
# La magnitud de la fuerza de Lennard-Jones
def lennard_jones(r, sigma, epsilon=1):
  fraccion = sigma**6/r**6
  return 4*epsilon*(12*fraccion**2-6*fraccion)/r

#### Numérico

In [12]:
# Función para generar pares de ángulo polar y azimutal
def generar_angulos():
  theta = 2*np.pi*np.random.random() # El ángulo polar
  phi = np.pi*np.random.random() # El ángulo azimutal
  return theta, phi

# Transformación de coordenadas esféricas a cartesianas
def transformar_coordenadas(theta, phi, magnitud):
  x = magnitud*np.sin(phi)*np.cos(theta)
  y = magnitud*np.sin(phi)*np.sin(theta)
  z = magnitud*np.cos(phi)
  return [x, y, z]

def calcular_distancia(x, y):
  return np.sqrt(((np.array(x)-np.array(y))**2).sum())

#### Generación

In [13]:
# Genera las posiciones de las partículas sin solaparse
def generar_posiciones(numero_particulas, tamanio, distribucion_r,
                       distribucion_m, distribucion_sigma):
  masas = []
  sigmas = []
  posiciones = []

  i = 0
  intentos = 0
  while i<numero_particulas:
    intentos += 1

    sigma_prueba = distribucion_sigma(0)

    theta_r, phi_r = generar_angulos()
    magnitud_r = tamanio*distribucion_r(0)
    r_prueba = np.array(transformar_coordenadas(theta_r, phi_r, magnitud_r))%tamanio

    coincide = True
    for j in range(len(posiciones)):
      distancia = calcular_distancia(r_prueba, posiciones[j])
      if distancia < sigma_prueba+sigmas[j]:
        coincide = False
        break
      
    if coincide:
      i += 1
      posiciones.append(r_prueba)
      sigmas.append(sigma_prueba)
      masas.append(distribucion_m(0))
      
    if intentos>numero_particulas**3:
      i = 0
      intentos = 0
      masas = []
      posiciones = []
  
  return sigmas, posiciones, masas

# Genera las velocidades de las partículas
def generar_velocidades(numero_particulas, m, kT, distribucion_v):
  velocidades = []
  for i in range(numero_particulas):
    theta_v, phi_v = generar_angulos()
    magnitud_v = distribucion_v(m, kT, 0)
    velocidades.append(transformar_coordenadas(theta_v, phi_v, magnitud_v))
  return velocidades

#### Simulación

In [14]:
# Generar la simulación
def iniciar_simulacion(numero_particulas=1000, tamanio = 1,
                       temperatura = 1, k_b = 1,
                       distribucion_r = lambda x: scipy.stats.uniform.rvs(0, 1),
                       distribucion_v = lambda m, kT, x: abs(maxwell_3d(m, kT).rvs()),
                       distribucion_m = lambda x: 1,
                       distribucion_sigma = lambda x: 0.001):
  
  # Primero se valida si es posible meter tantas esferas en una caja
  mean_sigma = np.array([distribucion_sigma(0) for i in range(100)]).mean() # El diámetro promedio
  densidad_empaquetamiento_max = np.pi/(3*np.sqrt(2))

  volumen_esferas = numero_particulas*(4*np.pi/3)*(mean_sigma/2)**3
  volumen_maximo = densidad_empaquetamiento_max*(tamanio**3)

  if volumen_esferas < volumen_maximo:
    velocidades = []

    sigmas, posiciones, masas = generar_posiciones(numero_particulas, tamanio,
                                                   distribucion_r, distribucion_m,
                                                   distribucion_sigma)

    mean_m = np.array(masas).mean()

    velocidades = generar_velocidades(numero_particulas, mean_m,
                                      k_b*temperatura, distribucion_v)
    
    simulacion = [[posiciones[i][0], posiciones[i][1], posiciones[i][2],
                   velocidades[i][0], velocidades[i][1], velocidades[i][2],
                   masas[i], sigmas[i], 0, 0, 0] for i in range(numero_particulas)]
    
    return simulacion

  
  else:
    print("No es posible meter tantas esferas en una caja de ese tamaño")
    return []

# Avanza la simulación dada una fuerza interna
def avanzar_simulacion(simulacion, dt=0.00001,
                       fuerza_interna = lambda r, sigma: lennard_jones(r, sigma, 1)):
  numero_particulas = len(simulacion)
  if numero_particulas>0:
    indices = range(numero_particulas)

    aceleraciones = [np.array([0, 0, 0]) for i in indices]

    for i, j in list(combinations(indices, 2)):
      r_i = np.array(simulacion[i][:3])
      r_j = np.array(simulacion[j][:3])

      r_hat = r_j-r_i
      r_hat = r_hat/calcular_distancia(r_hat, np.array([0, 0, 0]))

      distancia = calcular_distancia(r_i, r_j)
      sigma = (simulacion[i][7]+simulacion[j][7])/2

      fuerza_resultante = fuerza_interna(distancia, sigma)*r_hat

      aceleraciones[i] = aceleraciones[i] + fuerza_resultante/simulacion[i][6]
      aceleraciones[j] = aceleraciones[j] - fuerza_resultante/simulacion[j][6]

    simulacion_avanzada = simulacion[:]
    for i in indices:
      r_i = np.array(simulacion[i][:3])
      v_i = np.array(simulacion[i][3:6])
      a_i0 = np.array(simulacion[i][8:11])
      a_i1 = aceleraciones[i]

      simulacion_avanzada[i][:3] = r_i + v_i*dt + (1/2)*a_i0*(dt**2)
      simulacion_avanzada[i][3:6] = v_i + (1/2)*(a_i0+a_i1)*dt
      simulacion_avanzada[i][8:11] = a_i1
    
    return simulacion_avanzada
    
  else:
    print("No hay partículas en la simulación")

### Código

In [15]:
simulacion = iniciar_simulacion(numero_particulas=1000)[:]

In [16]:
simulacion_avanzada = avanzar_simulacion(simulacion, dt=0.001, fuerza_interna=lambda r, sigma: lennard_jones(r, sigma, 10))

In [17]:
simulacion

[[0.2306759010653312,
  0.31149420112587556,
  0.6303076119912021,
  0.00020988947946338346,
  1.006862644775628,
  0.866957883236449,
  1,
  0.001,
  -2.4946190820949327e-08,
  -1.7964533847317437e-08,
  6.1472446566326805e-09],
 [0.41558101449064666,
  0.6844193701375547,
  0.3743022254908982,
  1.449102066334427,
  0.833996294016117,
  -0.7981132596370828,
  1,
  0.001,
  2.716717149943605e-09,
  1.4473579575291415e-08,
  6.009953910141592e-09],
 [0.013752409880599899,
  0.04482685588513705,
  0.04169301191362633,
  0.39227520388603637,
  -0.622181714347658,
  0.4663650393905724,
  1,
  0.001,
  -8.581289783162193e-07,
  1.9477977823729575e-06,
  -2.3674453634587887e-07],
 [0.48570751537476675,
  0.7491979901674674,
  0.16529517266513447,
  -0.11664183412518961,
  0.9734126041236567,
  1.6879421562233572,
  1,
  0.001,
  2.4416657000153117e-09,
  -1.500296290866873e-08,
  6.659583210947383e-09],
 [0.7195284870130689,
  0.8700190730650313,
  0.353212533668609,
  0.5443116760227865,
 

In [18]:
simulacion_avanzada

[[0.2306759010653312,
  0.31149420112587556,
  0.6303076119912021,
  0.00020988947946338346,
  1.006862644775628,
  0.866957883236449,
  1,
  0.001,
  -2.4946190820949327e-08,
  -1.7964533847317437e-08,
  6.1472446566326805e-09],
 [0.41558101449064666,
  0.6844193701375547,
  0.3743022254908982,
  1.449102066334427,
  0.833996294016117,
  -0.7981132596370828,
  1,
  0.001,
  2.716717149943605e-09,
  1.4473579575291415e-08,
  6.009953910141592e-09],
 [0.013752409880599899,
  0.04482685588513705,
  0.04169301191362633,
  0.39227520388603637,
  -0.622181714347658,
  0.4663650393905724,
  1,
  0.001,
  -8.581289783162193e-07,
  1.9477977823729575e-06,
  -2.3674453634587887e-07],
 [0.48570751537476675,
  0.7491979901674674,
  0.16529517266513447,
  -0.11664183412518961,
  0.9734126041236567,
  1.6879421562233572,
  1,
  0.001,
  2.4416657000153117e-09,
  -1.500296290866873e-08,
  6.659583210947383e-09],
 [0.7195284870130689,
  0.8700190730650313,
  0.353212533668609,
  0.5443116760227865,
 